Generates the data

In [ ]:
from __future__ import division
import numpy as np
from numpy.random import rand

def symmetra(n):

    W=np.zeros((n,n))

    for i in range (n):
        for j in range (n):

            if i<j:
                W[i,j]=np.random.normal(0,1)
                W[j,i]=W[i,j]


    return W

def sigmoid(x):
    y=1/(1+(np.exp(-x)))
    return y


samp=1000
m=1000*samp#column, number of data vectors
n=64
#W=np.random.rand(64,64)
#b=np.random.rand(64,1)
W=symmetra(n)
b=np.zeros((64,1))

np.save('Wreal.dat',W)

X=np.zeros((n,m))
X[:,0]=np.random.randint(2, size=n)
print (X[:,0])
print(X.shape)

for j in range(0,m-1):

    for i in range(0,n):


        t=np.dot(W[i,:],X[:,j])+b[i]



        P1=sigmoid(t)
        #print (P1)
        #X[i, j + 1]=np.random.choice(2, 1, p=[1-P1,P1])
        X[i, j + 1] = np.random.binomial(1, P1, 1)





XX=np.zeros((n,m/samp))

for j in range(0,m):

    if j%samp==0:
        print(j)
        XX[:,j/samp]=X[:,j]

print (XX)
print (XX.shape)

Reads which system and imports directory accordingly

In [ ]:
import numpy as np
import socket



def imp(x):

    name = socket.gethostname()
 

    if name=='DESKTOP-H2QOB9B':

        data=np.load('C:/Users/Owner/Dropbox/sokovia/'+x)

    elif name=='DESKTOP-QJ171NH':
        data=np.load('C:/Users/Admin/Dropbox/sokovia/'+x)



    return data


Runs the Theano gradient descent on the MPF cost function

In [1]:
from __future__ import print_function
import matplotlib.pyplot as pp
import os
import sys
import timeit
import wisp
import numpy as np

import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams

def symmetra(n):

    W=np.zeros((n,n))

    for i in range (n):
        for j in range (n):

            if i<j:
                W[i,j]=np.random.normal(0,1)
                W[j,i]=W[i,j]


    return W


class dA(object):

    def __init__(self,input=None,W=None,b=None):

        n=64
        if not W:

            initial_W = np.asarray(symmetra(n),dtype=theano.config.floatX)

            W = theano.shared(value=initial_W, name='W', borrow=True)

        if not b:
            initial_b = np.asarray(np.zeros((64,10)),dtype=theano.config.floatX)
            b = theano.shared(value=initial_b, name='b', borrow=True)

        self.W = W
        self.b = b


        if input is None:

            self.x = T.dmatrix(name='input')
        else:
            self.x = input

        self.params = [self.W, self.b]


    def get_cost_updates(self, learning_rate):

        delt = 0.5 - self.x

        cost = 0.00000000001*T.sum( T.exp((2.0*delt * (T.dot(self.W,self.x)+self.b))) )

        gparams = T.grad(cost, self.params)



        updates = [
            (param, param - learning_rate * gparam)
            for param, gparam in zip(self.params, gparams)
        ]

        return (cost, updates)


def test_dA(learning_rate=0.1, training_epochs=100,
            batch_size=10):

    datasets = wisp.imp('DataX.dat.npy')
    print(datasets.shape)
    train_set_x = datasets
    train_set_x2 = theano.shared(value=train_set_x, name='train_set_x2', borrow=True)

    n_train_batches = train_set_x.shape[1] / batch_size
    n_train_batches=int(n_train_batches)

    index = T.lscalar()
    x = T.matrix('x')

    da = dA(input=x)

    cost, updates = da.get_cost_updates(learning_rate=learning_rate)

    train_da = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x2[:,index * batch_size: (index + 1) * batch_size]
        }
    )




    for epoch in range(training_epochs):
        c = []
        for batch_index in range(n_train_batches):
            c.append(train_da(batch_index))

        print('Training epoch %d, cost ' % epoch, np.mean(c))


    return da.W.get_value(borrow=True)


if __name__ == '__main__':

    Wmpf=test_dA()

    x,y=Wmpf.shape
    Wreal=wisp.imp('Wreal.dat.npy')
    print(Wmpf)
    mse=np.sum(np.square(Wreal-Wmpf))/(x*y)
    print(mse)
    np.save('Wmpf1.dat', Wmpf)
    WmpfLine = np.ravel(Wmpf)
    WrealLine = np.ravel(Wreal)
    pp.plot(WmpfLine[0:19])
    pp.plot(WrealLine[0:19])
    pp.show()


(64L, 1000L)
Training epoch 0, cost  0.00199715889472
Training epoch 1, cost  0.00141981037544
Training epoch 2, cost  0.00112859275588
Training epoch 3, cost  0.000946929888004
Training epoch 4, cost  0.000820710294346
Training epoch 5, cost  0.000727019990249
Training epoch 6, cost  0.000654275946092
Training epoch 7, cost  0.000595916618866
Training epoch 8, cost  0.000547915355656
Training epoch 9, cost  0.00050764942859
Training epoch 10, cost  0.000473329774334
Training epoch 11, cost  0.000443690074989
Training epoch 12, cost  0.000417806482893
Training epoch 13, cost  0.000394987759642
Training epoch 14, cost  0.000374705511511
Training epoch 15, cost  0.000356548324062
Training epoch 16, cost  0.000340190702186
Training epoch 17, cost  0.000325371490467
Training epoch 18, cost  0.000311878540751
Training epoch 19, cost  0.000299537601163
Training epoch 20, cost  0.000288204121785
Training epoch 21, cost  0.000277757115744
Training epoch 22, cost  0.000268094494707
Training epo